In [82]:
import csv
import pandas as pd

### Try importing csv file

In [76]:
pd.read_csv(r'./data/data_engineering_dataset.csv')

ParserError: Error tokenizing data. C error: Expected 17 fields in line 11388, saw 30


### Getting a parser error which suggests there are extra columns in a problematic line... let's investigate.

In [84]:
with open('./data/data_engineering_dataset.csv') as fd:
    reader=csv.reader(fd)
    interestingrows=[row for idx, row in enumerate(reader) if idx in (11387,)]
    print(interestingrows)

[['5GZCZ33D16S845647', '2006', 'Silver', 'SUV', '4 Cylinder', 'VUE                      ', '95136', 'Accurate', 'SATURN', 'Truck', '465', 'SUV', '2.2L', 'GasolinVIN', 'CarYear', 'Color', 'VehBody', 'EngineType', 'Make', 'Miles', 'Odometer', 'Brand', 'VehType', 'LocationNum', 'CarType', 'EngineLiters', 'FuelType', 'Transmission', 'SaleLoc', 'PurchVal']]


### It looks like this row had the columns values inserted into it... this is suspicious, let's investigate further.

In [86]:
with open('./data/data_engineering_dataset.csv') as fd:
    reader=csv.reader(fd)
    interestingrows=[row for idx, row in enumerate(reader) if idx in (11387*2,)]
    print(interestingrows)

[['5GZCZ33D16S845647', '2006', 'Silver', 'SUV', '4 Cylinder', 'VUE                      ', '95136', 'Accurate', 'SATURN', 'Truck', '465', 'SUV', '2.2L', 'Gasoline', 'Automatic', '465', '5500']]


### This appears to be a duplicate row... My suspicion is that there are more duplicate rows, perhaps duplicated data was accidentally inserted into this dataset file.

In [87]:
#Read the file with explicit columns to remove the problem from the extra cols
df = pd.read_csv(r'./data/data_engineering_dataset.csv', names = ['VIN',	'CarYear',	'Color',	'VehBody',	'EngineType',	'Make',	'Miles',	'Odometer',	'Brand',	'VehType',	
                                                                  'LocationNum',	'CarType',	'EngineLiters',
                                                                  'FuelType',	'Transmission',	'SaleLoc',	'PurchVal'], skiprows = 1)

In [98]:
#Split these rows in their own df and compare them.
df1 = df.iloc[0:11387]
df2 = df.iloc[11387:11387*2]

#Returns just the rows from the new df that differ from the source df.
def get_different_rows(source_df, new_df):
    merged_df = source_df.merge(new_df, indicator=True, how='outer')
    changed_rows_df = merged_df[merged_df['_merge'] == 'right_only']
    return changed_rows_df.drop('_merge', axis=1)

#Find differences.
get_different_rows(df1,df2)

,VIN,CarYear,Color,VehBody,EngineType,Make,Miles,Odometer,Brand,VehType,LocationNum,CarType,EngineLiters,FuelType,Transmission,SaleLoc,PurchVal
11387,5GZCZ33D16S845647,2006,Silver,SUV,4 Cylinder,VUE,95136.0,Accurate,SATURN,Truck,465,SUV,2.2L,Gasoline,Automatic,465,5500


### No differences found between these rows, I can comfortably assume that this is duplicated data and it should be removed from the dataset.

### Let's try ignoring this row to see if there are any other problems with the file.

In [99]:
pd.read_csv(r'./data/data_engineering_dataset.csv', skiprows=[0,11387])

ParserError: Error tokenizing data. C error: Expected 17 fields in line 22782, saw 20


In [113]:
with open('./data/data_engineering_dataset.csv') as fd:
    reader=csv.reader(fd)
    interestingrows=[row for idx, row in enumerate(reader) if idx in (22781,)]
    print(interestingrows)

[['1G4HP52K834149596', '2003', 'White', '4 Door', '6 Cylinder', 'LESABRE CUSTOM           ', '117896', 'Accurate', 'BUICK', 'Passenger', '391', 'Full-size', '3.8L', 'Gasoline', 'Automatic', '391', '5500e', 'Automatic', '465', '5500']]


### Looks like this could be another copy-paste error.. let's inspect these rows to find out more.

In [114]:
df.iloc[11387*2:]

,VIN,CarYear,Color,VehBody,EngineType,Make,Miles,Odometer,Brand,VehType,LocationNum,CarType,EngineLiters,FuelType,Transmission,SaleLoc,PurchVal
22774,2CNDL73FX66077410,2006,Gray,SUV,6 Cylinder,EQUINOX,109057.00000,Accurate,CHEVROLET,Truck,466,SUV,3.4L,Gasoline,Automatic,466,4480
22775,1G1AD5F56A7169546,2010,Red,4 Door,4 Cylinder,COBALT,84970.00000,Accurate,CHEVROLET,Passenger,481,Compact,2.2L,Gasoline,5-Speed,481,5000
22776,1G1AL58F787211970,2008,White,4 Door,4 Cylinder,COBALT,92982.00000,Accurate,CHEVROLET,Passenger,223,Compact,2.2L,Gasoline,Automatic,223,3800
22777,2FTRF17274CA55319,2004,Silver,Truck,6 Cylinder,F150 HERITAGE,95201.00000,Accurate,FORD,Truck,81,Full-Size Truck,4.2L,Gasoline,5-Speed,81,4800
22778,1FMCU03108KC39727,2008,Grey,SUV,6 Cylinder,ESCAPE XLT,115005.00000,Inaccurate,FORD,Truck,437,SUV,3.0L,Gasoline,Automatic,437,5740
22779,4A3AB26FX4E088213,2004,Black,4 Door,4 Cylinder,GALANT DE,80339.66667,Accurate,MITSUBISHI,Passenger,442,Mid-size,2.4L,Gasoline,Automatic,442,4400
22780,1G4HP52K834149596,2003,White,4 Door,6 Cylinder,LESABRE CUSTOM,117896.00000,Accurate,BUICK,Passenger,391,Full-size,3.8L,Gasoline,Automatic,391,5500e
22781,2CNDL73FX66077410,2006,Gray,SUV,6 Cylinder,EQUINOX,109057.00000,Accurate,CHEVROLET,Truck,466,SUV,3.4L,Gasoline,Automatic,466,4480
22782,1G1AD5F56A7169546,2010,Red,4 Door,4 Cylinder,COBALT,84970.00000,Accurate,CHEVROLET,Passenger,481,Compact,2.2L,Gasoline,5-Speed,481,5000
22783,1G1AL58F787211970,2008,White,4 Door,4 Cylinder,COBALT,92982.00000,Accurate,CHEVROLET,Passenger,223,Compact,2.2L,Gasoline,Automatic,223,3800


### More duplicated data... we can safely remove it from the dataset as well.

In [185]:
#Remove first set of duplicate data
cleaned_df = df.drop(df.index[0:11387])
#Reset index
cleaned_df.reset_index(inplace=True, drop=True)
#Remove second set of duplicated data
cleaned_df = cleaned_df.drop(df.index[11387:11394])
#Reset index
cleaned_df.reset_index(inplace=True, drop=True)
cleaned_df.head()

,VIN,CarYear,Color,VehBody,EngineType,Make,Miles,Odometer,Brand,VehType,LocationNum,CarType,EngineLiters,FuelType,Transmission,SaleLoc,PurchVal
0,1G1AL15F777226903,2007,Silver,2 Door,4 Cylinder,COBALT LT,117029.0,Accurate,CHEVROLET,Passenger,86,Compact,2.2L,Gasoline,Automatic,86,4900
1,5NPEU46C66H126152,2006,Silver,4 Door,4 Cylinder,SONATA GLS/LX,108890.0,Accurate,HYUNDAI,Passenger,189,Full-size,2.4L,Gasoline,Automatic,189,5700
2,3MEHM08Z48R665374,2008,White,4 Door,4 Cylinder,MILAN,138000.0,Accurate,MERCURY,Passenger,360,Full-size,2.3L,Gasoline,Automatic,360,2890
3,1HGCP2F81AA000524,2010,Red,4 Door,4 Cylinder,ACCORD,89643.0,Accurate,HONDA,Passenger,302,Mid-size,2.4L,Gasoline,Automatic,302,9000
4,2G2WC52C951325364,2005,Maroon,4 Door,8 Cylinder,GRAND PRIX,127806.0,Accurate,PONTIAC,Passenger,295,Full-size,5.3L,Gasoline,Automatic,295,4000


### Standard data cleaning

In [192]:
cleaned_df['Color'] = cleaned_df['Color'].str.upper()
cleaned_df['VIN'] = cleaned_df['VIN'].str.upper()
cleaned_df['Make'] = cleaned_df['Make'].str.rstrip()
cleaned_df['EngineLiters'] = cleaned_df['EngineLiters'].str.strip('L')

In [193]:
cleaned_df.to_csv('./data/data_engineering_dataset_cleaned.csv')